In [1]:
import numpy as np
import pandas as pd
from scipy.io import loadmat
from sklearn.metrics.pairwise import cosine_similarity
import matplotlib.pyplot as plt
import random

%matplotlib inline

In [2]:
def load_network(file_name):
    """
    TO-DO:
    
    rewrite this to accomodate for the working directory of user
    """
    path = '/Users/taikannakada/FASCINATE/' + file_name
    data = loadmat(path)
    return data


def layer_as_array(data, fascinate=False):
    '''
    once matlab data is loaded via scipy.io.loadmat
    convert layer data into matrix
    
    data: loaded matlab data
    fascinate: if True then edge_list for cos_sim matrix
    '''
    layers = []
    fascinate_layers = []
    
    if fascinate: # for layer Bi
        num_layers = data['F'].shape[1]
        for i in range(num_layers):
            fascinate_layers.append(data['F'][0,i][0][0][0])
        return fascinate_layers
    
    else: # for layer Ai
        num_layers = data['G'].shape[1]
        for i in range(num_layers):
            layers.append(data['G'][0,i][0][0][0].toarray())
        return layers
      
    
def edgelist_A(layer):
    """
    edgelist format for MultiVERSE:
    (layer, source, target, weight)
    """
    edges = np.transpose(np.nonzero(layer))
    ones = np.ones((edges.shape[0],1)).astype(int)
    new_edges = np.hstack((ones, edges, ones))
    return new_edges


def edgelist_B(layer):
    """
    edgelist format for MultiVERSE:
    (layer, source, target, weight)
    """
    mean = np.mean(layer)
    std = np.std(layer)
    threshold = mean + 3*std
    if threshold > 1:
        threshold = .99
    above_threshold = layer > threshold
    
    edges = np.transpose(np.where(above_threshold)).astype(int)
    ones = np.ones((edges.shape[0],1)).astype(int)
    twos = ones * 2
    new_edges = np.hstack((twos, edges, ones))
    return new_edges


def looped_cosine_similarity(fascinate_layers):
    """
    Cosine_similarity:
    Instead of computing the cos_sim between F and F, we can define a for loop:
    In each loop i, the cos_sim_i is computed beween F and F[i*N:(i+1)*N,], where
    N is the number of rows to be computed in each loop
    Shuo
    """
    N = 500
    cos_sims = []

    for layer in fascinate_layers:
        cos_sim_layer = []
        for i in range(int(layer.shape[0]/N)+1):
            if (i+1)*N > layer.shape[0]:
                final_idx = layer.shape[0]
            else:
                final_idx = (i+1)*N
            cos_sim_i = cosine_similarity(layer, layer[i*N:final_idx])
            cos_sim_layer.append(cos_sim_i)
        cos_sim_layer = np.concatenate(cos_sim_layer, axis=1)
        cos_sims.append(cos_sim_layer)
        
    return cos_sims


def create_multiplex(file_name):
    """
    create multiplex network for each layer of HMLN
    """
    network = load_network(file_name)
    
    # for Ai of multiplex network
    layers = layer_as_array(network)
    
    edges_A = []
    for layer in layers:
        edges_A.append(edgelist_A(layer))
        
    # for Bi of multiplex network
    fascinate_layers = layer_as_array(network, fascinate=True)
    cos_sims = looped_cosine_similarity(fascinate_layers)
    
    edges_B = []
    for each in cos_sims:
        edges_B.append(edgelist_B(each))
    
    # construct multiplex from Ai, Bi
    multiplex_networks = []
    for i in range(len(edges_A)):
        each_multiplex = np.vstack((edges_A[i], edges_B[i]))
        multiplex_networks.append(each_multiplex)
        
    return multiplex_networks

In [4]:
multiplex = create_multiplex('infra5.mat')

In [5]:
len(multiplex)

5

In [ ]:
######################################
# Save layers as .txt for MULTIVERSE #
######################################

for i in range(len(edges_A)):
    np.savetxt('infra5_edges_layer' + str(i+1) + '.txt', multiplex[i], fmt='%d', delimiter=' ') 

In [6]:
############################
# After running MULTIVERSE #
############################

infra_node_embeddings = []
"""
infra_node_embeddings[0]['1'] --> embedding of first node of first layer
"""
for i in range(len(multiplex)):
    layer_embed = np.load('/Users/taikannakada/MultiVERSE/ResultsMultiVERSE/infra5_edges_layer'+ str(i+1) + '.txtembeddings_M.npy', allow_pickle=True)
    embeddings = layer_embed[()]
    infra_node_embeddings.append(embeddings)

In [7]:
# # DU represents total cross-layer connections

# cross_layer_connections = [[2,3],[2,4],[3,5],[4,5],
#                            [1,2],[1,3],[1,4],[1,5]]

# DU = []        # connectivity matrix
# DU_edges = []  # (layer, source, target, layer)
# DU_labels = [] # link labels --> 1: positive, 0: negative

# infra5 = load_network('infra5.mat')
# num_layers = infra5['DU'].shape[1]

# for i in range(num_layers):
#     DU.append(infra5['DU'][0,i][0][0][0].toarray())
    
#     edges = np.transpose(np.nonzero(DU[i]))
#     ones = np.ones((edges.shape[0],1)).astype(int)
    
#     start = cross_layer_connections[i][0] * ones
#     end = cross_layer_connections[i][1] * ones
#     new_edges = np.hstack((start, edges, end))
    
#     DU_edges.append(new_edges)
#     DU_labels.append(ones)

In [8]:
######################
#This is the test set
######################

# DO represents 50% of the total cross-layer connections

cross_layer_connections = [[2,3],[2,4],[3,5],[4,5],
                           [1,2],[1,3],[1,4],[1,5]]

DO_TEST= []
DO_edges_TEST = []
DO_labels_TEST = [] # link labels --> 1: positive, 0: negative

num_layers = infra5['DO'].shape[1]

for i in range(num_layers):
    difference = infra5['DU'][0,i][0][0][0].toarray() - infra5['DO'][0,i][0][0][0].toarray()
    DO_TEST.append(difference)
    
    edges = np.transpose(np.nonzero(DO_TEST[i]))
    ones = np.ones((edges.shape[0],1)).astype(int)
    
    start = cross_layer_connections[i][0] * ones
    end = cross_layer_connections[i][1] * ones
    new_edges = np.hstack((start, edges, end))
    
    DO_edges_TEST.append(new_edges)
    DO_labels_TEST.append(ones)

In [9]:
#
# BELOW IS CODE FOR TRAINING:
#
# https://stellargraph.readthedocs.io/en/stable/demos/link-prediction/node2vec-link-prediction.html
#
# USING 'DO' OF FASCINATE
#
# 1. NEGATIVE SAMPLING OF DO --> PRODUCE AS MANY NEGATIVE SAMPLES AS POSITIVE
#
# 2. APPLY BINARY OPERATOR ON EMBEDDINGS OF SOURCE AD TARGET NODE OF EACH SAMPLED EDGE
#     [BINARY OPERATORS:   - HADAMARD
#                          - L1
#                          - L2
#                          - AVERAGE]
#
# 3. TRAIN LOGISTIC REGRESSION CLASSIFIER TO PREDICT BINARY VALUE (EDGE OR NOT)
#
# 4. EVALUATE PERFORMANCE

In [9]:
############################################################
# 1. Negative Sampling                                     #
#                                                          #
# Sampling as many negative samples as there are positive  #
#                                                          #
# NODES:                                                   #
# layer1: 39                                               #
# layer2: 151                                              #
# layer3: 61                                               #
# layer4: 47                                               #
# layer5: 51                                               #
#                                                          #
############################################################


for i in range(len(DO_edges_TEST)):
    '''
    negative sampling
    '''
    ones = np.ones((DO_edges_TEST[i].shape[0],1)).astype(int)
    zeros = np.zeros((DO_edges_TEST[i].shape[0],1)).astype(int)
    start = cross_layer_connections[i][0] * ones
    end = cross_layer_connections[i][1] * ones

    a = np.random.randint(DO_TEST[i].shape[0], size=(DO_edges_TEST[i].shape[0],1))
    b = np.random.randint(DO_TEST[i].shape[1], size=(DO_edges_TEST[i].shape[0],1))
    sampled_edges = np.hstack((start, a, b, end))

    DO_edges_TEST[i] = np.vstack((DO_edges_TEST[i], sampled_edges))
    DO_labels_TEST[i] = np.vstack((DO_labels_TEST[i], zeros))
    

In [10]:
for i in range(len(DO_edges_TEST)):
    if i == 0:
        all_cross_link = DO_edges_TEST[i]
        all_cross_link_labels = DO_labels_TEST[i]
    else:
        all_cross_link = np.vstack((all_cross_link, DO_edges_TEST[i]))
        all_cross_link_labels = np.vstack((all_cross_link_labels, DO_labels_TEST[i]))
        
all_cross_link_labels = all_cross_link_labels.reshape((all_cross_link_labels.shape[0],)) # shape: (128,)

In [22]:
#############################################################################################
# 2. APPLYING BINARY OPERATOR ON EMBEDDINGS OF SOURCE AND TARGET NODES OF EACH SAMPLED EDGE #
#############################################################################################


from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegressionCV
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import roc_auc_score


def operator_hadamard(u,v):
    return u*v

def operator_l1(u,v):
    return np.abs(u-v)

def operator_l2(u,v):
    return (u-v)**2

def operator_avg(u,v):
    return (u+v)/2.0

binary_operators = [operator_hadamard, operator_l1, operator_l2, operator_avg]


# embeddings of source and target nodes of sampled edge
def link_examples_to_features(infra_node_embeddings, all_cross_link, binary_operator):
    '''
    binary operator on source and target nodes
    
    start_layer = each[0]
    end_layer = each[3]
    start_node = each[1]+1 # 0-index --> node nummber is i+1
    end_node = each[2]+1   # 0-index --> node nummber is i+1
    '''
    return [
        binary_operator(infra_node_embeddings[each[0] - 1][str(each[1]+1)].reshape((128,)), infra_node_embeddings[each[3] - 1][str(each[2]+1)].reshape((128,)))
        for each in all_cross_link
    ]


#################################################################################
# 3. TRAIN LOGISTIC REGRESSION CLASSIFIER TO PREDICT BINARY VALUE (EDGE OR NOT) #
# 4. EVALUATE PERFORMANCE                                                       #
#################################################################################
        

def link_prediction_classifier(max_iter=10000):
    """
    Logistic Regression:
    Cs:       inverse fo regularization strength
    cv:       default cross-validation generator is stratified k-folds. cv is number of folds used
    scoring:  scoring function
    max_iter: max number of iterations of optimization algorithm
    
    Pipeline:
    steps: list of (name, transform) tuples (implementing fit/transform) that are chained
    """
    lr_clf = LogisticRegressionCV(Cs=10, cv=10, scoring='roc_auc', max_iter=max_iter)
    return Pipeline(steps=[("sc", StandardScaler()), ("clf", lr_clf)])
    
    
def train_link_prediction_model(infra_node_embeddings, all_cross_link, link_labels, binary_operator):
    """
    """
    clf = link_prediction_classifier()
    link_features = link_examples_to_features(infra_node_embeddings, all_cross_link, binary_operator)
    clf.fit(link_features, link_labels)
    return clf


def evaluate_link_prediction_model(clf, 
                                   infra_node_embeddings, 
                                   all_cross_link, 
                                   all_cross_link_labels, 
                                   binary_operator):
    
    link_features_test = link_examples_to_features(infra_node_embeddings, 
                                                   all_cross_link, 
                                                   binary_operator)
    score = evaluate_roc_auc(clf, link_features_test, all_cross_link_labels)
    return score
    
    
def evaluate_roc_auc(clf, link_features, link_labels):
    predicted = clf.predict_proba(link_features)
    
    # check which class corresponds to positive links
    positive_column = list(clf.classes_).index(1)
    return roc_auc_score(link_labels, predicted[:, positive_column])


def run_link_prediction(binary_operator):
    clf = train_link_prediction_model(infra_node_embeddings, 
                                      all_cross_link, 
                                      all_cross_link_labels, 
                                      binary_operator)
    
    score = evaluate_link_prediction_model(clf, 
                                           infra_node_embeddings, 
                                           all_cross_link, 
                                           all_cross_link_labels, 
                                           binary_operator)
    
    return {
        "classifier": clf,
        "binary_operator": binary_operator,
        "score": score,
    }

In [18]:
#############
# RUN MODEL #
#############

results = [run_link_prediction(operator) for operator in binary_operators]
best_result = max(results, key=lambda result: result['score'])

print(f"Best result from '{best_result['binary_operator'].__name__}'")

pd.DataFrame(
    [(result['binary_operator'].__name__, result['score']) for result in results], 
    columns=('name', 'ROC AUC score'),
    ).set_index('name')

Best result from 'operator_hadamard'


,ROC AUC score
name,
operator_hadamard,0.941607
operator_l1,0.622542
operator_l2,0.736965
operator_avg,0.836762


In [23]:
test_score = evaluate_link_prediction_model(
    best_result["classifier"],
    infra_node_embeddings,
    all_cross_link,
    all_cross_link_labels,
    best_result["binary_operator"],
)
print(
    f"ROC AUC score on test set using '{best_result['binary_operator'].__name__}': {test_score}"
)

ROC AUC score on test set using 'operator_hadamard': 0.9416066481994461
